# Example: Centroid Vetting.
### How to look for centroid offset between in and out of transit
In this example we show how to use the vetting module to visualize the transit and take the difference between the in transit and out of transit to determine if there is a centroid offset between the two.

In [ ]:
import numpy
import matplotlib
import exovetter as exo
from exovetter import const
from exovetter import utils
import exovetter.vetters as vet
import lightkurve as lk
import astropy.units as u

In [ ]:
#Name of our first TOI Example
candidate = "TOI 1013.01"

### Use a utility function `get_mast_tce` to get the properties of the candidate and put into a tce object.

In [ ]:
tce = utils.get_mast_tce(candidate)
tce

### Grab the light curve using lightkurve

In [ ]:
lc = lk.search_lightcurve(candidate, sector=33, exptime=120).download()
lc.plot()

In [ ]:
#Flatten the light curve to remove some of the variability
flatlc = lc.flatten(window_length=501)
flatlc.plot()

## Visualize the transits 
Use `vet.VizTransits` to plot individual transits as well as the folded and smoothed transits.

In [ ]:
#Initiate the vetter
viz = vet.VizTransits()

#Run the vetter on the tce and lightcurve of your choice.
viz.run(tce[0], flatlc, plot=True)

### Zoom in on the transit

In [ ]:
viz = vet.VizTransits(transit_only=True)
viz.run(tce[0], flatlc, plot=True)

## Run the Centroid Vetter
The Centroid Vetter is run in the same way as the VizTransit method. It returns the average offset and significance that the event is on the target.
You get several plots showing the out of transits, in transit, and difference image. The centroids of each are measured using a Gaussian.
For the OOT and In-Transit, the centroid is constrained to the central +/-1 pixel in box row and column direction.
As input, the Centroid Vetter requires a Target Pixel File objects from lightkurve.

This example shows no significant offset between the out of transit location and the difference image centroid.

In [ ]:
#First we need to download the Target Pixel Information.
#We use a lightkurve method for this
tpf = lk.search_targetpixelfile(candidate, sector=33, exptime=120).download()
cent = vet.Centroid()
cent.run(tce[0],tpf, plot=True)

# Example 2.  An example with a centroid offset.
This time we will define our own TCE from scratch.
Here we use a qlp FFI light curve and use TESSCut to cutout the FFIs to get the target pixel file.
Note the significance that is returned is the significance that the difference image location is the same as the out of transit location, i.e. the significance that the transit occurs on the star in question.

In [ ]:
from exovetter import const as exo_const
from exovetter.tce import Tce

def get_qlp_lk(target, sector, author = "qlp", mission = "TESS", size = 11):
    """Returns lc and tpf for a given sector"""
    
    lc = lc = lk.search_lightcurve(target, sector=sector,
                              cadence="ffi",author=author).download()
    tpf = lk.search_tesscut(target, sector = sector).download(cutout_size = size)
    
    return lc, tpf

In [ ]:
mytce = Tce(target_name = "TIC 159573320",
            sector = 14,
            period = 10.8766 * u.day,
           epoch = 1687.4658 * u.day,
           epoch_offset =  exo_const.btjd,
           depth = .006943255 * exo_const.frac_amp,
           duration = 4 * u.h)

lc, tpf = get_qlp_lk(mytce['target_name'], mytce['sector'])

In [ ]:
viz = vet.VizTransits(transit_only=True, smooth=11)
viz.run(mytce, lc, plot=True)

In [ ]:
cent = vet.Centroid()
result = cent.run(mytce, tpf, plot=True)
print("Offset in pixels: %f " % result['offset'])

In [ ]:
print(offset)

## Example 3. A third example based on a TOI with a possible offset.


In [ ]:
candidate = "TOI 565.01"
tce = utils.get_mast_tce(candidate)
tce

In [ ]:
lc = lk.search_lightcurve(candidate)[0].download()
lc.plot()

In [ ]:
viz = vet.VizTransits(max_transits=8)
viz.run(tce[0],lc, plot=True, )

In [ ]:
#Examine difference images and measure centroids
tpf = lk.search_targetpixelfile(candidate)[0].download()
cent = vet.Centroid()
cent.run(tce[0],tpf,plot=True)